In [1]:
from datetime import datetime, timedelta
from pathlib import Path

import itertools
import matplotlib
from vivarium import Artifact, InteractiveContext
import ipywidgets
import pandas as pd, numpy as np
import seaborn as sns
pd.set_option('display.max_rows', 8)

The purpose of this notebook is to verify:

1. if the age-specific state table relative risks are equivalent to the pipeline relative risks
2. if the pipeline relative risk updates appropriately upon transition from the early to late neonatal age group
3. if the pipeline relative risk value does not change across timesteps other than for the age group transition

In [2]:
path = Path('/ihme/homes/alibow/vivarium_ciff_sam/src/vivarium_ciff_sam/model_specifications/ciff_sam.yaml')
path

PosixPath('/ihme/homes/alibow/vivarium_ciff_sam/src/vivarium_ciff_sam/model_specifications/ciff_sam.yaml')

In [ ]:
# manually updated the following parameters in the model specification file because it threw an error otherwise
    # population_size=100000
    # age_end=0.0767123287671233
    # exit_age=0.0767123287671233

sim = InteractiveContext(Path(path))

2022-01-13 11:08:13.147 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering metrics.1.population_manager.metrics as modifier to metrics
2022-01-13 11:08:13.167 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:65 - Running simulation from artifact located at /ihme/costeffectiveness/artifacts/vivarium_ciff_sam/ethiopia.hdf.
2022-01-13 11:08:13.168 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:66 - Artifact base filter terms are ['draw == 0'].
2022-01-13 11:08:13.169 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:67 - Artifact additional filter terms are None.
/ihme/code/alibow/miniconda/envs/vivarium_ciff_sam/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

2022-01-13 11:08:19.642 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering measles.incidence_rate.1.adjust_target as modifier to measles.incidence_rate
2022-01-13 11:08:19.645 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering measles.incidence_rate.paf.1.lookup_table_34 as modifier to measles.incidence_rate.paf
2022-01-13 11:08:19.912 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering lower_respiratory_infections.incidence_rate.1.adjust_target as modifier to lower_respiratory_infections.incidence_rate
2022-01-13 11:08:19.913 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering lower_respiratory_infections.incidence_rate.paf.1.lookup_table_36 as modifier to lower_respiratory_infections.incidence_rate.paf
2022-01-13 11:08:19.914 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline child_stunting.propensity
2022-01-13 11:08:19.9

2022-01-13 11:08:28.146 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering mortality_rate.5.riskstate.moderate_acute_malnutrition.adjust_mortality_rate as modifier to mortality_rate
2022-01-13 11:08:28.686 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline moderate_acute_malnutrition_to_severe_acute_malnutrition.transition_rate
2022-01-13 11:08:28.688 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline moderate_acute_malnutrition_to_severe_acute_malnutrition.transition_rate.paf
2022-01-13 11:08:29.038 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline moderate_acute_malnutrition_to_mild_child_wasting.transition_rate
2022-01-13 11:08:29.039 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline moderate_acute_malnutrition_to_mild_child_wasting.transition_rate.paf
2022-01-13 11:08:2

2022-01-13 11:08:31.841 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline risk_factor.severe_acute_malnutrition_treatment.exposure_parameters
2022-01-13 11:08:31.842 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline moderate_acute_malnutrition_treatment.propensity
2022-01-13 11:08:31.843 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline moderate_acute_malnutrition_treatment.exposure
2022-01-13 11:08:32.006 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline risk_factor.moderate_acute_malnutrition_treatment.exposure_parameters
2022-01-13 11:08:32.008 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline sq_lns.propensity
2022-01-13 11:08:32.009 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline sq_lns.coverage
2022-0

## Capture birthweight at initialization
For the early and late neonatal age groups

In [ ]:
pop0 = sim.get_population()
pop0

In [98]:
data = pd.concat([pop0.loc[:, ['age','sex',
                              'effect_of_low_birth_weight_and_short_gestation_on_late_neonatal_diarrheal_diseases_relative_risk',
                              'effect_of_low_birth_weight_and_short_gestation_on_early_neonatal_diarrheal_diseases_relative_risk']],
                 sim.get_value('low_birth_weight_and_short_gestation.exposure')(pop0.index),
                 sim.get_value('low_birth_weight.exposure')(pop0.index),
                 sim.get_value('effect_of_low_birth_weight_and_short_gestation_on_diarrheal_diseases.relative_risk')(pop0.index),
                 sim.get_value('short_gestation.exposure')(pop0.index)], 
                 axis=1).rename(columns={0:'relative_risk'})
data['age_group'] = np.where(data.age<7/365,'early_neonatal',
                           np.where(data.age<28/365,'late_neonatal',
                                   np.where(data.age<1,'post_neonatal', '1_to_5')))
data

,age,sex,effect_of_low_birth_weight_and_short_gestation_on_late_neonatal_diarrheal_diseases_relative_risk,effect_of_low_birth_weight_and_short_gestation_on_early_neonatal_diarrheal_diseases_relative_risk,risk_factor.low_birth_weight_and_short_gestation_exposure,low_birth_weight.exposure,relative_risk,short_gestation.exposure,age_group
0,0.044994,Male,4.244653,4.861793,cat40,2580.395737,4.244653,36.164988,late_neonatal
1,0.046464,Male,1.019543,1.000728,cat52,3448.290620,1.019543,38.995680,late_neonatal
2,0.052629,Female,2.288050,1.696371,cat42,2926.178834,2.288050,37.375198,late_neonatal
3,0.075327,Female,1.000000,1.000000,cat54,3681.270909,1.000000,39.985003,late_neonatal
...,...,...,...,...,...,...,...,...,...
99996,0.057286,Female,1.399476,1.151351,cat51,3125.716861,1.399476,41.798819,late_neonatal
99997,0.044341,Female,1.058347,1.000000,cat51,3457.984768,1.058347,41.059247,late_neonatal
99998,0.061990,Male,3.934176,3.564963,cat44,2530.101153,3.934176,38.800016,late_neonatal
99999,0.022806,Male,2.013787,1.455536,cat48,3033.461668,2.013787,37.664627,late_neonatal


In [99]:
enn = data.loc[data.age_group=='early_neonatal']
assert np.all(enn.effect_of_low_birth_weight_and_short_gestation_on_early_neonatal_diarrheal_diseases_relative_risk 
              == enn.relative_risk)==True

In [100]:
lnn = data.loc[data.age_group=='late_neonatal']
assert np.all(lnn.effect_of_low_birth_weight_and_short_gestation_on_late_neonatal_diarrheal_diseases_relative_risk 
              == lnn.relative_risk)==True

In [139]:
data_t = data.copy()
for x in list(range(0,10)):
    sim.step()
    data_t_minus_1 = data_t.copy()
    pop_t = sim.get_population()
    data_t = pd.concat([pop_t.loc[:, ['age','sex',
                              'effect_of_low_birth_weight_and_short_gestation_on_late_neonatal_diarrheal_diseases_relative_risk',
                              'effect_of_low_birth_weight_and_short_gestation_on_early_neonatal_diarrheal_diseases_relative_risk']],
                 sim.get_value('low_birth_weight_and_short_gestation.exposure')(pop_t.index),
                 sim.get_value('low_birth_weight.exposure')(pop_t.index),
                 sim.get_value('effect_of_low_birth_weight_and_short_gestation_on_diarrheal_diseases.relative_risk')(pop_t.index),
                 sim.get_value('short_gestation.exposure')(pop_t.index)], 
                 axis=1).rename(columns={0:'relative_risk'})
    
    enn_t = data_t.loc[data_t.age<7/365].index
    lnn_t = data_t.loc[data_t.age>7/365].index
    enn_t_minus_1 = data_t_minus_1.loc[data_t_minus_1.age<7/365].index
    lnn_t_minus_1 = data_t_minus_1.loc[data_t_minus_1.age>7/365].index

    assert np.all(data_t.loc[[c for c in lnn_t if c in enn_t_minus_1]].relative_risk 
                 == data_t.loc[[c for c in lnn_t if c in enn_t_minus_1]].effect_of_low_birth_weight_and_short_gestation_on_late_neonatal_diarrheal_diseases_relative_risk),"error1: relative risks did not update to LNN value upon age groups change correctly"
    assert np.all(data_t_minus_1.loc[[c for c in lnn_t if c in enn_t_minus_1]].relative_risk 
                 == data_t_minus_1.loc[[c for c in lnn_t if c in enn_t_minus_1]].effect_of_low_birth_weight_and_short_gestation_on_early_neonatal_diarrheal_diseases_relative_risk),"error2: issue with early neonatal RRs before age group change"
    assert np.all(data_t.loc[[c for c in enn_t if c in enn_t_minus_1]].relative_risk
                 == data_t_minus_1.loc[[c for c in enn_t if c in enn_t_minus_1]].relative_risk),"error3: relative risk changes within ENN age group"
    assert np.all(data_t.loc[[c for c in lnn_t if c in lnn_t_minus_1]].relative_risk
                 == data_t_minus_1.loc[[c for c in lnn_t if c in lnn_t_minus_1]].relative_risk),"error4: relative risk changes within LNN age group"
    enn = data_t.loc[data_t.age<7/365]
    lnn = data_t.loc[data_t.age>7/365]
    assert np.all(enn.relative_risk==enn.effect_of_low_birth_weight_and_short_gestation_on_early_neonatal_diarrheal_diseases_relative_risk), 'error5: ENN state table RR does not equal pipeline value'
    assert np.all(lnn.relative_risk==lnn.effect_of_low_birth_weight_and_short_gestation_on_late_neonatal_diarrheal_diseases_relative_risk),'error6: LNN state table RR does not equal pipeline value'

2022-01-12 11:55:59.490 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-02 00:00:00
2022-01-12 11:57:07.940 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-02 12:00:00
2022-01-12 11:58:17.964 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-03 00:00:00
2022-01-12 11:59:34.573 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-03 12:00:00
2022-01-12 12:00:51.243 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-04 00:00:00
2022-01-12 12:02:11.146 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-04 12:00:00
2022-01-12 12:03:18.809 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-05 00:00:00
2022-01-12 12:04:47.519 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-05 12:00:00
2022-01-12 12:06:10.573 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-06 00:00:00
2022-01-12 12:07:46.045 | DEBUG    | vivarium.framework.engine:step:142 - 2022-01-06 12:00:00
